In [1]:
import numpy as np
from scipy.linalg import expm

In [2]:
T = 2
d = 2
n = 2
steps = 100
b_t = 1.05

In [3]:
gamma = np.ones(d) * 1
beta = np.ones(d) * 1
X_bar= np.ones(d) * 0.04

Z0 = 1
S0 = np.array([[1, 1]])
X0 = np.array([[0.04, 0.04]])

def theta(X, beta, X_bar, gamma):
    res = []
    for i in range(n):
        res += [beta[i]*(X_bar[i] - X[0,i])**gamma[i]]
    return np.array(res)

In [4]:
A = np.matrix(np.identity(2))
B = np.matrix([[0.04,0.04]])
C = np.matrix(np.identity(2)*0.04)
sigma_t = np.identity(2)*0.1

In [5]:
def simulation(A, B, C, Z0, X0, sigma_t,S0, steps, T, d=2):
    h = T / steps
    dw = np.random.normal(0, np.sqrt(h), [d, steps])
    Z = Z0
    X = X0
    S = S0
    X_list = X0
    Z_list = np.array([Z])
    r_list = np.array([X[0,0]])
    S_list = S0
    for i in range(0, steps):
        dX = (np.dot(A,X.T).T+B) * h + np.dot(C,dw[:,i])
        X = X + dX
        dZ = np.dot(-theta(X,beta,X_bar,gamma), dw[:,i])
        Z = Z + Z * dZ
        dS = np.dot( sigma_t ,(theta(X,beta,X_bar,gamma) * h + dw[:,i])) + X[0,0] * h
        S = S + np.multiply(S, dS)
        X_list = np.concatenate((X_list,X),axis=0)
        Z_list = np.append(Z_list, Z)
        r_list = np.append(r_list, X[0,0])
        S_list = np.concatenate((S_list,S),axis=0)
    return X_list, Z_list, r_list, S_list, dw

In [6]:
X_list, Z_list, r_list, S_list, dw=simulation(A, B, C, Z0, X0,sigma_t ,S0,steps, T,d)

In [7]:
delta = np.array([0.5, 0.5])
k = np.array([0.02, 0.02])

In [8]:
alpha = np.ones(d) * 0.5

In [9]:
def H_1(X,k, delta):
    maxfunc = lambda x: max(x,0)
    x_new =  np.vectorize(maxfunc)(X-k)
    h = sum(delta* x_new)
    return h

In [10]:
def H_2(X,alpha=np.ones(d) * 0.5):
    return np.exp(np.dot(alpha, X))

In [11]:
pi_H = np.ones(d) * 0.5
rho = 2

In [12]:
def H_3(X,pi_H,rho):
    X_log = np.log(X)
    re = sum(pi_H* X_log)
    return re**rho
H_3(np.array([0.05,0.05]),pi_H,rho)

8.974411854812963

In [13]:
def ZB_1(r_list, Z_T, h):
    b_T = np.exp(sum(r_list[:-1])*h)
    return Z_T/b_T


In [14]:
def ZHB_1(X_T, r_list, Z_T, h, k, delta):
    H_T = H_1(X_T, k, delta)
    b_T = np.exp(sum(r_list[:-1])*h)
    return H_T*Z_T/b_T

In [15]:
def ZHB_2(X_T, r_list, Z_T, h):
    H_T = H_2(X_T)
    b_T = np.exp(sum(r_list[:-1])*h)
    return H_T*Z_T/b_T

In [16]:
def ZHB_3(X_T, r_list, Z_T, h,pi_H,rho):
    H_T = H_3(X_T,pi_H,rho)
    b_T = np.exp(sum(r_list[:-1])*h)
    return H_T*Z_T/b_T
ZHB_3(np.array([[0.05,0.05]]), r_list, 1, T/steps,pi_H,rho)

array([1.14407604, 1.14407604])

In [17]:
def MD_ZHB_1(X_list, r_list, Z_list, dw, A, C, T, steps, beta, X_bar, gamma, delta,k):
    # data type for A, C are matrix
    X_T = np.array(X_list[-1]).reshape(d,)
    Z_T = Z_list[-1]
    h = T/steps
    zhb = ZHB_1(X_T, r_list, Z_T, h, k, delta)
    zb= ZB_1(r_list, Z_T, h)
    ex_TA = expm(T*A)
    
    part_1 = np.linalg.inv(A)*(ex_TA*C - C)
    part_1 = np.array(part_1[0]).reshape(d,)
    
    

    part_2 = 0
    theta_t = np.array([beta[j]*(X_bar[j]-X_list[0,:][0,j])**gamma[j] for j in range(d)])
    for i in range(steps):
        v = h*i
        X_i = X_list[i,:]
        theta_i =  np.array([beta[j]*(X_bar[j]-X_i[0,j])**gamma[j] for j in range(d)])
        dw_i = dw[:,i]
        integrator = theta_i*h + dw_i
        derivative = np.array([beta[j]*gamma[j]*(X_bar[j]-X_i[0,j])**(gamma[j]-1) for j in range(d)])
        integrand = np.multiply(derivative.reshape(d,1),expm(v*A)*C)
        
        part_2 += np.dot(integrator, integrand)
    part_2 = np.array(part_2).reshape(d,)
    
    x_new = X_T-k>0
    part_3 = sum(delta * x_new * np.dot(ex_TA,C))

    return np.array(-zhb*(theta_t + part_1 + part_2) + zb *part_3)


In [18]:
MD_ZHB_1(X_list, r_list, Z_list, dw, A, C, T, steps, beta, X_bar, gamma, delta,k)

array([[-0.02134124,  0.10131504]])

In [19]:
def MD_ZHB_2(X_list, r_list, Z_list, dw, A, C, T, steps, beta=np.ones(d), X_bar = np.ones(d)*0.04, gamma=np.ones(d), alpha = np.ones(d)*0.5):
    # data type for A, C are matrix
    X_T = np.array(X_list[-1]).reshape(d,)
    Z_T = Z_list[-1]
    h = T/steps
    zhb = ZHB_2(X_T, r_list, Z_T, h)
    ex_TA = expm(T*A)
    part_1 = np.dot(np.dot(alpha,ex_TA),C)
    part_1 = np.array(part_1).reshape(d,)
    #print(part_1)
    part_2 = np.linalg.inv(A)*(ex_TA*C - C)
    part_2 = np.array(part_2[0]).reshape(d,)
    #print(part_2)
    part_3 = 0
    theta_t = np.array([beta[j]*(X_bar[j]-X_list[0,:][0,j])**gamma[j] for j in range(d)])
    for i in range(steps):
        v = h*i
        dw_i = dw[:,i]
        X_i = X_list[i,:]
        theta_i = np.array([beta[j]*(X_bar[j]-X_i[0,j])**gamma[j] for j in range(d)])
        integrator = theta_i*h + dw_i
        derivative = np.array([beta[j]*gamma[j]*(X_bar[j]-X_i[0,j])**(gamma[j]-1) for j in range(d)])
        integrand = np.multiply(derivative.reshape(d,1),expm(v*A)*C)
    
        part_3 += np.dot(integrator, integrand)
    part_3 = np.array(part_3).reshape(d,)
    #print(part_3)
    return zhb*(part_1 - theta_t - part_2 - part_3)

In [20]:
MD_ZHB_2(X_list, r_list, Z_list, dw, A, C, T, steps, beta=np.ones(d), X_bar = np.ones(d)*0.04, gamma=np.ones(d), alpha = np.ones(d)*0.5)

array([-0.10749442,  0.24301527])

In [21]:
def MD_ZHB_3(X_list, S_list,r_list, Z_list, dw, A, C, T, steps, sigma_t = np.identity(2)*0.1,beta=np.ones(d), X_bar =np.ones(d)*0.04, gamma=np.ones(d), pi_H=np.ones(d) * 0.5, rho=2):
    # data type for A, C are matrix
    X_T = np.array(X_list[-1]).reshape(d,)
    S_T = np.array(S_list[-1]).reshape(d,)
    Z_T = Z_list[-1]
    h = T/steps
    zhb = ZHB_3(S_T, r_list, Z_T, h,pi_H,rho)
    zb= ZB_1(r_list, Z_T, h)
    ex_TA = expm(T*A)
    
    S_log = np.log(S_T)
    re = sum(pi_H* S_log)
    scalar_1 = rho*re**(rho-1)
    
    
    ex_TA = expm(T*A)
    part_2 = np.linalg.inv(A)*(ex_TA*C - C)
    part_2 = np.array(part_2[0]).reshape(d,)
    
    part_3 = 0
    for j in range(d):
        inte_1 = sigma_t[:,j]
        
        for i in range(steps):
            sigma_jt = sigma_t[:,j]

            v = h*i
            dw_i = dw[:,i]
            X_i = X_list[i,:]
            integrator = sigma_t[:,j]
            derivative = np.array([beta[k]*gamma[k]*(X_bar[k]-X_i[0,k])**(gamma[k]-1) for k in range(d)])
            integrand = np.multiply(derivative.reshape(d,1),expm(v*A)*C)

            inte_3 = h * np.dot(integrator, integrand)
        
        part_3 += pi_H[j] * (inte_1 + part_2 + inte_3)
    part_3 = np.array(part_3).reshape(d,)
    
    part_4 = 0
    theta_t = np.array([beta[j]*(X_bar[j]-X_list[0,:][0,j])**gamma[j] for j in range(d)])
    for i in range(steps):
        v = h*i
        dw_i = dw[:,i]
        X_i = X_list[i,:]
        theta_i = np.array([beta[j]*(X_bar[j]-X_i[0,j])**gamma[j] for j in range(d)])
        integrator = theta_i*h + dw_i
        derivative = np.array([beta[j]*gamma[j]*(X_bar[j]-X_i[0,j])**(gamma[j]-1) for j in range(d)])
        integrand = np.multiply(derivative.reshape(d,1),expm(v*A)*C)
    
        part_4 += np.dot(integrator, integrand)
    part_4 = np.array(part_4).reshape(d,)
    
    return zb * scalar_1 * part_3 - zhb* (theta_t + part_2 + part_4)
MD_ZHB_3(X_list, S_list,r_list, Z_list, dw, A, C, T, steps,sigma_t = np.identity(2)*0.1, beta=np.ones(d), X_bar =np.ones(d)*0.04, gamma=np.ones(d), pi_H=np.ones(d) * 0.5, rho=2)

array([0.15378069, 0.05584893])

In [23]:
def pi_t_1(M, A, B, C, Z0, X0, S0, steps, T,b_t,beta, X_bar, gamma, sigma_t,k,d):
    h = T/steps
    theta_t = np.array([beta[j]*(X_bar[j]-X0[0,j])**gamma[j] for j in range(d)])
    sigma_t_inv_T = np.linalg.inv(sigma_t).T
    pi_i = 0
    for i in range(M):
        X_list, r_list, Z_list, S_list, dw = simulation(A, B, C, Z0, X0, sigma_t,S0, steps, T, d=2)
        X_T = np.array(X_list[-1]).reshape(d,)
        Z_T = Z_list[-1]
        zhb = ZHB_1(X_T, r_list, Z_T, h, k, delta)
        MD_zhb = MD_ZHB_1(X_list, r_list, Z_list, dw, A, C, T, steps, beta, X_bar, gamma, delta,k)
        pi_i += np.dot(sigma_t_inv_T, (zhb*theta_t+MD_zhb).T)
    
    pi_i = pi_i/(M*b_t)
    return pi_i   

In [24]:
def pi_t_2(M, A, B, C, Z0, X0, S0, steps, T, b_t, beta=np.ones(d), X_bar = 0.04*np.ones(d), gamma=np.ones(d), alpha = np.ones(d)*0.5, sigma_t=np.identity(d)*0.1):
    h = T/steps
    theta_t = np.array([beta[j]*(X_bar[j]-X0[0,j])**gamma[j] for j in range(d)])
    sigma_t_inv_T = np.linalg.inv(sigma_t).T
    pi_i = 0
    for i in range(M):
        X_list, r_list, Z_list, S_list, dw = simulation(A, B, C, Z0, X0, sigma_t,S0, steps, T, d=2)
        X_T = np.array(X_list[-1]).reshape(d,)
        Z_T = Z_list[-1]
        zhb = ZHB_2(X_T, r_list, Z_T, h)
        MD_zhb = MD_ZHB_2(X_list, r_list, Z_list, dw, A, C, T, steps, beta, X_bar, gamma,alpha)
        pi_i += np.dot(sigma_t_inv_T, zhb*theta_t+MD_zhb)
    pi_i = pi_i/(M*b_t)
    return pi_i   

In [25]:
def pi_t_3(M, A, B, C, Z0, X0, S0, steps, T, b_t, beta=np.ones(d), X_bar = 0.04*np.ones(d), gamma=np.ones(d), pi_H=np.ones(d) * 0.5, rho=2, sigma_t=np.identity(d)*0.1):
    h = T/steps
    theta_t = np.array([beta[j]*(X_bar[j]-X0[0,j])**gamma[j] for j in range(d)])
    sigma_t_inv_T = np.linalg.inv(sigma_t).T
    pi_i = 0
    for i in range(M):
        X_list, r_list, Z_list, S_list, dw = simulation(A, B, C, Z0, X0, sigma_t,S0, steps, T, d=2)
        X_T = np.array(X_list[-1]).reshape(d,)
        Z_T = Z_list[-1]
        S_T = np.array(S_list[-1]).reshape(d,)
        zhb = ZHB_3(S_T, r_list, Z_T, h,pi_H, rho)
        MD_zhb = MD_ZHB_3(X_list, S_list, r_list, Z_list, dw, A, C, T, steps, sigma_t,beta, X_bar, gamma, pi_H, rho)
        pi_i += np.dot(sigma_t_inv_T, zhb*theta_t+MD_zhb)
    pi_i = pi_i/(M*b_t)
    return pi_i

In [30]:
M=1000
b_t = 1.05
pi_t_2(M, A, B, C, Z0, X0, S0, steps, T, b_t)

array([-0.06332805,  0.23305424])

In [31]:
pi_t_1(M, A, B, C, Z0, X0, S0, steps, T,b_t,beta, X_bar, gamma, sigma_t,k,d)

array([[0.02642656],
       [0.1058337 ]])

In [32]:
pi_t_3(M, A, B, C, Z0, X0, S0, steps, T, b_t, beta=np.ones(d), X_bar = 0.04*np.ones(d), gamma=np.ones(d), pi_H=np.ones(d) * 0.5, rho=2, sigma_t=np.identity(d)*0.1)

array([0.11808776, 0.03030447])